# Chat Bot Project 
it is based on NLP and deep learning by using tensor flow made by google to make the model that classify the tag and randomely choose a response based on the user input 

#Json File Generation 
that contain the dectionary of the tags and to improve the chat bot to answer more questions in different tags we can add more tag or use different approach to build the chat bot as use api 

In [1]:
import json

aDict = {"intents": [
        {"tag": "greeting",
         "patterns": ["Hi there", "How are you", "Is anyone there?","Hey","Hola", "Hello", "Good day"],
         "responses": ["Hello", "Good to see you again", "Hi there, how can I help?"],
         "context": [""]
        },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],
         "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],
         "context": [""]
        },
        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "That's helpful", "Awesome, thanks", "Thanks for helping me"],
         "responses": ["My pleasure", "You're Welcome"],
         "context": [""]
        },
        {"tag": "query",
         "patterns": ["What is big bang?"],
         "responses": ["The Big Bang theory is the prevailing cosmological model explaining the existence of the observable universe from the earliest known periods through its subsequent large-scale evolution."],
         "context": [""]
        } 
    ]
 }
jsonString = json.dumps(aDict)
jsonFile = open("data.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

#Import Needed Liberaries 

In [2]:
!pip install tflearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=58b98be0e440a6daa2929666e00cbe7a407c1de88d96c1aed467763dc23587f2
  Stored in directory: /root/.cache/pip/wheels/4a/d5/f8/9585b4a100c0fd73da204ee785457d67c85e1b9050f009a849
Successfully built tflearn


In [3]:
import nltk 
from nltk.stem.lancaster import LancasterStemmer
import random
import numpy as np 
import json 
import pickle 
import tensorflow as tf 
import tflearn
nltk.download('punkt')

Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Implement the Chat Bot 

In [4]:
with open('/content/data.json') as intents:
  data = json.load(intents)
# open the json file and store it in data 

In [5]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['My pleasure', "You're Welcome"],
   'context': ['']},
  {'tag': 'query',
   'patterns': ['What is big bang?'],
   'responses': ['The Big Bang theory is the prevailing cosmological model explaining the existence of the observable universe from the earliest known periods through its subsequent large-scale evolution.'],
   'context': ['']}]}

In [6]:
stemmer = LancasterStemmer()
words = []
labels = []
x_docs = []
y_docs = []
# lists to store the data from json  file 

In [7]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        x_docs.append(wrds)
        y_docs.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [8]:
# Stemming the words and removing duplicate elements.
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)


In [9]:
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

# One hot encoding, Converting the words to numerals
for x, doc in enumerate(x_docs):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)


    output_row = out_empty[:]
    output_row[labels.index(y_docs[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)

# using a bag of words (from scratch) to make a training features 

In [10]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 1499  | total loss: 0.07990 | time: 0.007s
| Adam | epoch: 500 | loss: 0.07990 - acc: 1.0000 -- iter: 16/18
Training Step: 1500  | total loss: 0.08100 | time: 0.011s
| Adam | epoch: 500 | loss: 0.08100 - acc: 1.0000 -- iter: 18/18
--


# Prepare the Input Function for predict 

In [11]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for s_word in s_words:
        for i, w in enumerate(words):
            if w == s_word:
                bag[i] = 1

    return np.array(bag)
# function to convert the input of the user to the bag of word to easly pass to the model for prediction 

In [12]:
def chat_bot():

    while True:
        inp = input("\n\nYou : ")
        if inp.lower() == 'exit':
            break

    #Porbability of correct response 
        results = model.predict([bag_of_words(inp, words)])

    # Picking the greatest number from probability
        results_index = np.argmax(results)

        tag = labels[results_index]

        for tg in data['intents']:

            if tg['tag'] == tag:
                responses = tg['responses']
                print("\nBot:\t" + random.choice(responses))

# try the chat bot 

In [13]:
chat_bot()



You : hello

Bot:	Hi there, how can I help?


You : what is the big bang 

Bot:	The Big Bang theory is the prevailing cosmological model explaining the existence of the observable universe from the earliest known periods through its subsequent large-scale evolution.


You : thanks

Bot:	You're Welcome


You : bye

Bot:	Bye! Come back again soon.


You : exit


the future work is to change the approach of the chat bot and use api in addition to make a GUI to be user friendly 